In [27]:
import helpy
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize

In [66]:
class build_training_data:
    def __init__(self, image_file, frame_data, number_test = 5,training_shape = 32, upscale_size = 4):
        self.image_file = image_file
        self.frame_data = frame_data
        self.training_shape = training_shape
        self.centralize = training_shape // 2
        self.upscale_size = upscale_size
        self.number_test = number_test
            
    def prep_image(self, width = 2):
        im = plt.imread(self.image_file).astype(float)
        s = width*im.std()
        m = im.mean()
        im -= m - s
        im /= 2 * s
        np.clip(im,0,1,out=im)
        return im
    
    def necessary_data(self):
        pos = self.frame_data['xy'] - np.array([self.centralize, self.centralize])
        pos_start = pos.astype(int)
        pos_cen = (self.frame_data['xy'] - pos_start)*self.upscale_size
        pos_arrow = pos_cen + self.upscale_size * 15.0 * np.array([np.cos(frame_data['o']), np.sin(frame_data['o'])]).T
        pos_arrow = pos_arrow.astype(np.float32)
        pos_cen = pos_cen.astype(np.float32)
        return pos_start, pos_cen, pos_arrow
    
    def interactive_mode(self):
        cv2.namedWindow("Build", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Build", 400,400)
        pi2 = np.pi * 2
        img = self.prep_image()
        pos_start, pos_cen, pos_arrow = self.necessary_data()
        training_regress= list()
        training_class = list()
        training_x = list()
        for frame, (x_start,y_start),arr_cen, arr_pos in zip(self.frame_data['o'][:self.number_test],pos_start[:self.number_test],pos_cen[:self.number_test],pos_arrow[:self.number_test]):
            temp_img = img[x_start:x_start+self.training_shape, y_start:y_start + self.training_shape]
            resize_img = resize(temp_img, (self.training_shape*self.upscale_size, self.training_shape*self.upscale_size))
            if np.isnan(arr_pos[0]):
                cv2.imshow("Build", resize_img)
            else:
                arrow_plot = cv2.arrowedLine(resize_img, (arr_cen[1],arr_cen[0]),(arr_pos[1], arr_pos[0]),(255,0,0),2)
                cv2.imshow('Build', resize_img)
            print "detect or is:", frame
            press = cv2.waitKey(0)
            # everything is good, store current information into training sets
            if press == 49:
                training_x.append(temp_img)
                training_regress.append(frame)
                training_class.append(1)
            # enter plot mode
            elif press == 50: 
                points_store = []
                button_down = False
                cv2.imshow("Build", resize_img)
                def orientation_click(event, x, y, flags, param):
                    global button_down
                    if event == cv2.EVENT_LBUTTONUP and button_down:
                        button_down = False
                        points_store.append((x,y))
                        cv2.arrowedLine(resize_img, points_store[0], points_store[1], (255,0,0),1)
                        cv2.imshow("Build", resize_img)
                        
                    elif event == cv2.EVENT_MOUSEMOVE and button_down:
                        image = resize_img.copy()
                        cv2.arrowedLine(image, points_store[0], (x,y), (255,0,0),1)
                        cv2.imshow("Build", image)

                    elif event == cv2.EVENT_LBUTTONDOWN and len(points_store) < 2:
                        button_down = True
                        points_store.insert(0, (x,y))
                cv2.setMouseCallback('Build',orientation_click, points_store)
                enter = cv2.waitKey(0)
                orientation = np.arctan2(points_store[1][0]-points_store[0][0], points_store[1][1]-points_store[0][1])
                orientation = (orientation + pi2)%pi2
                print("Click orientation is: {}".format(orientation))
                if enter == 10:
                    training_x.append(temp_img)
                    training_regress.append(orientation)
                    training_class.append(1)
            elif press == 51:
                training_x.append(temp_img)
                training_regress.append(np.nan)
                training_class.append(0)
            elif press == 27:
                break
        cv2.destroyAllWindows()
        for i in range (1,5):
            cv2.waitKey(1)
        return training_x, training_regress, training_class

In [68]:
class1 = build_training_data(im_file, frame_data)
a = class1.interactive_mode()

detect or is: 3.837428
Click orientation is: 3.84779174602
detect or is: 2.3602843
Click orientation is: 2.37173727992
detect or is: 2.3842254
Click orientation is: 2.31956167097
detect or is: 2.2701907
Click orientation is: 2.4099062195
detect or is: nan
Click orientation is: 0.626136718218


In [62]:
a[2]

[1, 1, 1, 1, 1]

In [3]:
im_file='/home/zhejun/Result/20181203/result_00001.tif'

In [4]:
data =helpy.load_data('/home/zhejun/Result/20181203/result')
data['o'] = (data['o'] + np.pi)%(2*np.pi)
frame_data = data[data['f']==0]

In [25]:
class1 = build_training_data(im_file, frame_data)

In [87]:
class T:
    a = 3
    def __init__(self):
        self.b = []
    
    def test(self):
        a = 4
        for i in range(10):
            self.b.append(i)
        print self.b
        print a
        print self.a

In [88]:
tt = T()

In [89]:
tt.test()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4
3


In [90]:
tt.b

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
b = list(a)

In [22]:
b.append(np.random.rand(3,3))

In [23]:
b

[array([[0.12007083, 0.31236005, 0.39983972],
        [0.20795855, 0.26741788, 0.79793932],
        [0.24240908, 0.84201298, 0.79312201]]),
 array([[0.23329187, 0.93620566, 0.79562605],
        [0.22787315, 0.38248369, 0.96826021],
        [0.21167955, 0.46250039, 0.86538129]]),
 array([[0.78900815, 0.37224248, 0.28179681],
        [0.18699984, 0.07279788, 0.38413059],
        [0.25596358, 0.32216125, 0.90411345]]),
 array([[0.81235322, 0.89896871, 0.92387773],
        [0.39337494, 0.21870837, 0.73681468],
        [0.37939043, 0.29513041, 0.34079612]])]